In [1]:
from requests_oauthlib import OAuth1Session
import json
import datetime, time, sys
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import os
from pykakasi import kakasi

In [2]:
CK = '4qzQk73Qw5aJfAQ9dcrmWXkdm'                             # Consumer Key
CS = '9o3yAxnEnkXOnFGGMdFTkTCDM6g6FjLt5RqNJAfzEq2KF8U59W'    # Consumer Secret
AT = '115662117-P1nN3CHlh4bQhrWUMtHiz1xt5oYVt9eg8Ra7yzLq'    # Access Token
AS = 'RrRIYr7l2EtyMQ5qMYIMM9FmxFxEK28pxR6pqRVES5oWX'         # Accesss Token Secert

In [3]:
class TweetsGetter(object):
    __metaclass__ = ABCMeta
 
    def __init__(self):
        self.session = OAuth1Session(CK, CS, AT, AS)
 
    @abstractmethod
    def specifyUrlAndParams(self, keyword):
        '''
        呼出し先 URL、パラメータを返す
        '''
 
    @abstractmethod
    def pickupTweet(self, res_text, includeRetweet):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
 
    @abstractmethod
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
 
    def collect(self, total = -1, onlyText = False, includeRetweet = False):
        '''
        ツイート取得を開始する
        '''
 
        #----------------
        # 回数制限を確認
        #----------------
        self.checkLimit()
 
        #----------------
        # URL、パラメータ
        #----------------
        url, params = self.specifyUrlAndParams()
        params['include_rts'] = str(includeRetweet).lower()
        # include_rts は statuses/user_timeline のパラメータ。search/tweets には無効
 
        #----------------
        # ツイート取得
        #----------------
        cnt = 0
        unavailableCnt = 0
        while True:
            res = self.session.get(url, params = params)
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)
 
                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue
 
            unavailableCnt = 0
 
            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)
 
            tweets = self.pickupTweet(json.loads(res.text))
            if len(tweets) == 0:
                # len(tweets) != params['count'] としたいが
                # count は最大値らしいので判定に使えない。
                # ⇒  "== 0" にする
                # https://dev.twitter.com/discussions/7513
                break
 
            for tweet in tweets:
                if (('retweeted_status' in tweet) and (includeRetweet is False)):
                    pass
                else:
                    if onlyText is True:
                        yield tweet['text']
                    else:
                        yield tweet
 
                    cnt += 1
                    if cnt % 100 == 0:
                        print ('%d件 ' % cnt)
 
                    if total > 0 and cnt >= total:
                        return
 
            params['max_id'] = tweet['id'] - 1
 
            # ヘッダ確認 （回数制限）
            # X-Rate-Limit-Remaining が入ってないことが稀にあるのでチェック
            if ('X-Rate-Limit-Remaining' in res.headers and 'X-Rate-Limit-Reset' in res.headers):
                if (int(res.headers['X-Rate-Limit-Remaining']) == 0):
                    self.waitUntilReset(int(res.headers['X-Rate-Limit-Reset']))
                    self.checkLimit()
            else:
                print ('not found  -  X-Rate-Limit-Remaining or X-Rate-Limit-Reset')
                self.checkLimit()
 
    def checkLimit(self):
        '''
        回数制限を問合せ、アクセス可能になるまで wait する
        '''
        unavailableCnt = 0
        while True:
            url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
            res = self.session.get(url)
 
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)
 
                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue
 
            unavailableCnt = 0
 
            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)
 
            remaining, reset = self.getLimitContext(json.loads(res.text))
            if (remaining == 0):
                self.waitUntilReset(reset)
            else:
                break
 
    def waitUntilReset(self, reset):
        '''
        reset 時刻まで sleep
        '''
        seconds = reset - time.mktime(datetime.datetime.now().timetuple())
        seconds = max(seconds, 0)
        print ('\n     =====================')
        print ('     == waiting %d sec ==' % seconds)
        print ('     =====================')
        sys.stdout.flush()
        time.sleep(seconds + 10)  # 念のため + 10 秒
 
    @staticmethod
    def bySearch(keyword):
        return TweetsGetterBySearch(keyword)
 
    @staticmethod
    def byUser(screen_name):
        return TweetsGetterByUser(screen_name)
 

In [4]:
class TweetsGetterBySearch(TweetsGetter):
    '''
    キーワードでツイートを検索
    '''
    def __init__(self, keyword):
        super(TweetsGetterBySearch, self).__init__()
        self.keyword = keyword
        
    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/search/tweets.json'
        params = {'q':self.keyword, 'count':100}
        return url, params
 
    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text['statuses']:
            results.append(tweet)
 
        return results
 
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['search']['/search/tweets']['remaining']
        reset     = res_text['resources']['search']['/search/tweets']['reset']
 
        return int(remaining), int(reset)

In [5]:
def save_tweet(word, conv, since, until):
    user = []
    created_at = []
    text = []
    retweet = []
    key = []
    latitude = []
    longitude = []

    # キーワードで取得
    keyword = word + ' AND -filter:replies AND since:' + since + ' AND until:' + until
    getter = TweetsGetter.bySearch(keyword)

    # ユーザーを指定して取得 （screen_name）
    #getter = TweetsGetter.byUser('AbeShinzo')

    cnt = 0
    for tweet in getter.collect(total = 1000000, includeRetweet=True):
        #cnt += 1
        #print ('------ %d' % cnt)
        #print ('{} {} {}'.format(tweet['id'], tweet['created_at'], '@'+tweet['user']['screen_name']))
        #print (tweet['text'])
        user.append(tweet['user']['screen_name'])
        created_at.append(tweet['created_at'])
        text.append(tweet['text'])
        
        if tweet['coordinates']:
            longitude.append(tweet['coordinates']['coordinates'][0])
            latitude.append(tweet['coordinates']['coordinates'][1])
        else:
            longitude.append(np.nan)
            latitude.append(np.nan)
            
        if 'retweeted_status' in tweet.keys():
            retweet.append(True)
        else:
            retweet.append(False)
        
        key.append(word)
    
    user = Series(user)
    created_at = Series(created_at)
    text = Series(text)
    retweet = Series(retweet)
    key = Series(key)
    longitude = Series(longitude)
    latitude = Series(latitude)

    #各シリーズをデータフレーム化
    df = pd.concat([user, created_at, text, retweet, key, longitude, latitude],axis=1)

    #カラム名
    df.columns=['user', 'created_at','text', 'RT', "key", "longitude", "latitude"]

    #csvファイルとして保存
    df.to_csv('tweet/' + until + '/' + conv.do(word) + '_' + until + '.csv', sep = '\t',encoding='utf-16')
    
    return df

In [6]:
Kakasi = kakasi()

Kakasi.setMode('H', 'a')
Kakasi.setMode('K', 'a')
Kakasi.setMode('J', 'a')

conv = Kakasi.getConverter()

In [7]:
today = datetime.date.today()
before = today - datetime.timedelta(7)
until = str(today)
since = str(before)
print(since)

2018-05-28


In [8]:
words = ["八幡浜", 
         "マーマレード大会",
         "どーや市場",
         "どーや食堂",
         "日土小学校",
         "アゴラマルシェ", 
         "もっきんろーど", 
         "諏訪崎",
         "平家谷", 
         "みなと湯", 
         "ゆめみかん", 
         "はまぽん",
         "飯田市",
         "飯田線秘境駅",
         "飯田水引",
         "下栗の里",
         "遠山郷",
         "りんご並木",
         "しらびそ高原",
         "野底山森林公園"]

In [9]:
newpath = "tweet/" + until 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [10]:
for word in words:
    print('current word:', word)
    _ = save_tweet(word, conv, since, until)
#ID_df = save_tweet('飯田市', 'Iida', since, until)

current word: 八幡浜
100件 
200件 
300件 
400件 
500件 
600件 
700件 
800件 
current word: マーマレード大会
current word: どーや市場
current word: どーや食堂
current word: 日土小学校
100件 
current word: アゴラマルシェ
current word: もっきんろーど
current word: 諏訪崎
current word: 平家谷
current word: みなと湯
current word: ゆめみかん
current word: はまぽん
current word: 飯田市
100件 
200件 
current word: 飯田線秘境駅
current word: 飯田水引
current word: 下栗の里
current word: 遠山郷
current word: りんご並木
current word: しらびそ高原
current word: 野底山森林公園
